<a href="https://colab.research.google.com/github/Mschaid/nma_dl_project/blob/main/notebooks/nma_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dependencies
[things vision repo](https://github.com/ViCCo-Group/THINGSvision)

In [ ]:

$ pip install --upgrade thingsvision
$ pip install git+https://github.com/openai/CLIP.git

In [7]:
import numpy as np
import pandas as pd
import torch

from IPython.display import Image

# Background

**Revealing the multidimensional mental representations of natural objects underlying human similarity judgements**: [link](https://www.nature.com/articles/s41562-020-00951-3.epdf?sharing_token=nHVvIjL4RlwottzTvjgItNRgN0jAjWel9jnR3ZoTv0M2M6rn_U7HFAne1ZfilQzEBRHexV_lH_PgbXIY9yiNp7yXnn3qOHdywYiFVOWZQIy_pJjjNv1vFLYrZ6nmAuWmC3u_Qh81SlZMMBe4PI2qBw31vRsjHdk2XJntcS9fkMc%3D )

**Human alignment of neural network representations**: [link](https://arxiv.org/abs/2211.01201)

**What can 1.8 billion regressions tell us about the pressures shaping high-level visual representation in brains and machines?**: [link](https://www.biorxiv.org/content/10.1101/2022.03.28.485868v2)

# Research questions
- Do different kinds of neural network architectures better align with human behavioral representations? (e.g., [2])
- Need better control for some models (see [3])
- Can we improve the alignment between network and human representations? (i.e., learn a simple - linear transformation W as in [2])?
- Can we replace W with a simple non-linear transformation (e.g., small NN) to further improve alignment?


From the second to last layer (negotiable) we get the 1xn (image x number of neurons) and construct an image x neurons matrix

Calculate the distance between between images in representational space 
	assume the closer the images the more correlated they are

we label the decision correct or wrong based on what humans chose 

The goal is to find a W ( in the image below) that complies with human decision
note on the image : while visually 3 seems furter away 2 is circled(squared) just for the sake of example


In [10]:
Image(url = "https://github.com/Mschaid/nma_dl_project/blob/main/images/Screenshot%202023-07-13%20at%208.40.25%20AM.jpg?raw=true")

## Data resources: 
### [thing](https://osf.io/jum2f/)

### [odd one out](https://osf.io/f5rn6/)


#### Triples data from: [triples data csv](https://osf.io/h2smy)

In [16]:
data = pd.read_csv('https://osf.io/download/h2smy/', delim_whitespace=True)
data.head()

,image1,image2,image3,choice,RT,noise_ceiling,subject_id,HIT_nr,trial_nr,age,gender,date,time,dataset
0,1245,1050,494,3,14457,0,BMWLG5ZY79VLA,1,1,NaN,other,2018-02-05,09:31:16,1.0
1,888,1788,1250,3,5043,0,BMWLG5ZY79VLA,1,2,NaN,other,2018-02-05,09:31:16,1.0
2,1256,734,946,2,6605,0,BMWLG5ZY79VLA,1,3,NaN,other,2018-02-05,09:31:16,1.0
3,1069,1037,953,1,6177,0,BMWLG5ZY79VLA,1,4,NaN,other,2018-02-05,09:31:16,1.0
4,1693,803,963,3,2327,0,BMWLG5ZY79VLA,1,5,NaN,other,2018-02-05,09:31:16,1.0


In [13]:
data.describe()

,image1,image2,image3,choice,RT,noise_ceiling,HIT_nr,trial_nr,age,dataset
count,4.699160e+06,4.699160e+06,4.699160e+06,4.699160e+06,4.699160e+06,4.699160e+06,4.699160e+06,4.699160e+06,3.259600e+06,4.036680e+06
mean,9.272810e+02,9.277512e+02,9.273038e+02,1.998373e+00,4.440737e+03,5.755731e-02,1.192657e+05,1.050000e+01,4.588138e+01,3.466101e+00
std,5.351330e+02,5.352359e+02,5.353955e+02,8.125337e-01,1.012596e+04,3.629220e-01,6.952256e+04,5.766282e+00,9.342258e+01,1.127249e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-3.601371e+06,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00
25%,4.640000e+02,4.640000e+02,4.640000e+02,1.000000e+00,1.984000e+03,0.000000e+00,5.876100e+04,5.750000e+00,3.100000e+01,4.000000e+00
50%,9.270000e+02,9.280000e+02,9.270000e+02,2.000000e+00,2.846000e+03,0.000000e+00,1.186815e+05,1.050000e+01,3.900000e+01,4.000000e+00
75%,1.390000e+03,1.391000e+03,1.391000e+03,3.000000e+00,4.530000e+03,0.000000e+00,1.797530e+05,1.525000e+01,5.200000e+01,4.000000e+00
max,1.854000e+03,1.854000e+03,1.854000e+03,3.000000e+00,1.511196e+06,3.000000e+00,2.396470e+05,2.000000e+01,7.140000e+03,4.000000e+00


In [14]:
data.isnull().sum()

image1                 0
image2                 0
image3                 0
choice                 0
RT                     0
noise_ceiling          0
subject_id             0
HIT_nr                 0
trial_nr               0
age              1439560
gender                 0
date                   0
time                   0
dataset           662480
dtype: int64